# Persistence and Efficency
You now might be aware of some of the problems with parsing several pages and how that can quicly get out of hand. While parsing, you might chose to persist the collected data, so that it can be analyzed and cleaned later. 
Parsing, retrieving, saving, and cleaning data are all separate actions, and you shouldn't try to work with data while collecting. In this notebook you'll practice further parsing techniques along with persistency, both for JSON and CSV formats as well as using SQL and a database.

Start by loading one of the available HTML files into the `scrapy` library

**Note**: This notebook relies on several dependencies that must be pre-installed in your environment. If you haven't done so, please run the following command in your terminal or command prompt:
```bash
python3 -m venv venv
venv/bin/pip install -r requirements.txt
```

In [131]:
import scrapy
import os
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url, 'r', encoding='utf-8') as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')

In [132]:
# Make sure that the interesting data is available 
table = response.xpath('//table')[1].xpath('tbody')
for tr in table.xpath('tr'):
    print(tr.xpath('td[1]/b/text()').extract()[0],
          tr.xpath('td[2]/a/text()').extract()[0],
         
    )

Gold Steve Smith
Silver Tim Forsyth
Bronze Takahiro Kimino


This interaction with `scrapy` in a Jupyter Notebook is useful because you don't need to run the special shell and you also don't need to run the whole spider. Once you learn what you need to do here, you can adapat the spider to persist data.
First, start by persisting data as JSON. To do this, you will need to keep the information in a Python data structure like a dictionary, and then load it as a JSON object, and finally, save it to a file.

In [133]:
scrapped_data = {}
for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    scrapped_data[medal] = athlete

scrapped_data

{'Gold': 'Steve Smith', 'Silver': 'Tim Forsyth', 'Bronze': 'Takahiro Kimino'}

In [134]:
import json

# You can convert Python into JSON first, but there is no need if you use `json.dump()`
# as shown next
json_data = json.dumps(scrapped_data)

# Persist it in a file:
with open("1992_results.json", "w") as _f:
    # use dump() with the Python dictionary directly. 
    # the conversion is done on the fly
    json.dump(scrapped_data, _f)

Now that you can persist the scrapped data as JSON, you can also use CSV. This is specially useful if you want to to some data science operations. Although you can use an advanced library like Pandas for this, you can use the standard library CSV module from Python.

In [135]:
# construct the data first

column_names = ["Medal", "Athlete"]
rows = []

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    rows.append([medal, athlete])


In [136]:
# Now persist it to disk
import csv

with open("1992_results.csv", "w") as _f:
    writer = csv.writer(_f)

    # write the column names
    writer.writerow(column_names)

    # now write the rows
    writer.writerows(rows)

Finally, you can persist data to a database. Unlike the JSON and CSV approach, using a database is much more memory efficient. This is the principal reason why you want to use a database instead of a file on disk. Imagine capturing 10GB of data. This could potentially mean that you need 10GB of available memory to hold onto that data before saving it to disk.
By using a database, you can save the data as the data is gathered. 

For the next cells, use a SQLite database to persist the data. Create the file-based database and the table needed.

In [138]:
import sqlite3
connection = sqlite3.connect("1992_results.db")
db_table = 'CREATE TABLE IF NOT EXISTS results (id integer primary key, medal TEXT, athlete TEXT)'
cursor = connection.cursor()
cursor.execute(db_table)
connection.commit()

In [141]:
# Now it is time to persist the data. Open the connection again
connection = sqlite3.connect("1992_results.db")
cursor = connection.cursor()
query = 'INSERT INTO results(medal, athlete) VALUES(?, ?)'

for tr in table.xpath('tr'):
    medal = tr.xpath('td/b/text()').extract()[0]
    athlete = tr.xpath('td/a/text()').extract()[0]
    cursor.execute(query, (medal, athlete)) 
    connection.commit()

The data is now persisted in a file-based database that you can query. Verify that all works by creating a new connection and querying the database.

Update the _wikipedia_ project and spider to use some of these techniques to persist data. Next, try parsing all the files in the _html_ directory instead of just one and persist all results. Do you think you can parse other information as well? 

Try parsing the height and the results from all the other athletes, not just the top three places.

In [142]:
#verify by querying the databasee
connection = sqlite3.connect("1992_results.db")
cursor = connection.cursor()
query = 'select * from results'
cursor.execute(query) 
rows = cursor.fetchall()
for row in rows:
    print(row)
connection.commit()

(1, 'Gold', 'Steve Smith')
(2, 'Silver', 'Tim Forsyth')
(3, 'Bronze', 'Takahiro Kimino')


In [143]:
#Parsing all the files in html directory 
#Create a new database called results.db and table high_jump_results
import scrapy
import os
import sqlite3
from pathlib import Path

# Set up database connection
conn = sqlite3.connect("results.db")
cur = conn.cursor()

# Create table if not exists
cur.execute("""
CREATE TABLE IF NOT EXISTS high_jump_results (
    rank TEXT,
    athlete TEXT
)
""")
conn.commit()



In [144]:
# Get all HTML files under 'html' directory and parsing througt all files

html_dir = Path('html')
html_files = list(html_dir.glob("*"))  # All .html files in 'html/' folder

for html_file in html_files:
    with open(html_file, 'r', encoding='utf-8') as f:
        url_data = f.read()

    # Create Scrapy TextResponse for each file
    response = scrapy.http.TextResponse(str(html_file), body=url_data, encoding='utf-8')

   # Parse the table (adjust index [1] if the table index is different)
    table = response.xpath('//table')[1].xpath('tbody')
    for tr in table.xpath('tr'):
        try:
            rank = tr.xpath('td/b/text()').extract()[0].strip()
            #print(rank)
            athlete = tr.xpath('td/a/text()').extract()[0].strip()
            #medal = tr.xpath('td/b/text()').extract()[0]
            #athlete = tr.xpath('td/a/text()').extract()[0]
            # Insert data into the database
            cur.execute("INSERT INTO high_jump_results (rank, athlete) VALUES (?, ?)", (rank, athlete))
        except IndexError:
            # Skip rows where expected data is missing
            #print("no data")
            continue

# Commit and close the connection after processing all files
conn.commit()
conn.close()

In [48]:
#verify by querying the databasee
conn = sqlite3.connect("results.db")
cur = conn.cursor()
query = 'select * from high_jump_results'
cur.execute(query) 
rows = cur.fetchall()
for row in rows:
    print(row)
conn.commit()

('Gold', 'Steve Smith')
('Silver', 'Tim Forsyth')
('Bronze', 'Takahiro Kimino')
('Gold', 'Jagan Hames')
('Silver', 'Antoine Burke')
('Bronze', 'Mika Polku')
('Gold', 'Mark Boswell')
('Silver', 'Svatoslav Ton')
('Silver', 'Ben Challenger')
('Gold', 'Alfredo Deza')
('Silver', 'Yin Xueli')
('Bronze', 'Aleksandr Veryutin')


In [145]:
# paring height information from 1992 record
import scrapy
import os
import sqlite3

current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url, 'r', encoding='utf-8') as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')
table1 = response.xpath('//table')[3].xpath('tbody')
for tr in table1.xpath('tr'):
    try:
        rank = tr.xpath('td/text()').extract_first()
        name = tr.xpath('td/a/text()').extract_first()
        height = tr.xpath('td/b/text()').extract_first()       

        print(rank,name,height)

    except AttributeError:
        # This handles missing elements gracefully (e.g., header rows)
        #print("Wrong data")
        continue

None None None
1 Tim Forsyth 2.16
1 Takahiro Kimino 2.16
3 Xu Xiaodong 2.16
3 Clifford van Reed 2.16
5 Sven Ootjers 2.16
6 Stanley Osuide 2.13
6 Kristofer Lamos 2.13
6 Antoine Burke 2.13
6 Coenraad Roux 2.13
10 Kim Tae-hoi 2.13
11 Dejan Miloševic 2.13
12 Kostas Liapis 2.13
13 Hugo Muñoz 2.13
14 Giorgio Florindi 2.05
15 Oskari Frösén 2.05
16 Ignacio Pérez 2.00


In [146]:
#Generate a new table height in 1992_results.db database
import sqlite3
import scrapy
import os

connection = sqlite3.connect("1992_jump_results.db")
db_table = 'CREATE TABLE IF NOT EXISTS heights (id integer primary key, rank TEXT, athlete TEXT, height TEXT)'
cursor = connection.cursor()
cursor.execute(db_table)
connection.commit()

In [147]:
current_dir = os.path.abspath('')
url = os.path.join(current_dir, "html/1992_World_Junior_Championships_in_Athletics_–_Men's_high_jump")
with open(url, 'r', encoding='utf-8') as _f:
    url_data = _f.read()

response = scrapy.http.TextResponse(url, body=url_data, encoding='utf-8')
table1 = response.xpath('//table')[3].xpath('tbody')
for tr in table1.xpath('tr'):
    try:
        rank = tr.xpath('td/text()').extract_first()
        name = tr.xpath('td/a/text()').extract_first()
        height = tr.xpath('td/b/text()').extract_first()       
        print(rank,name,height)
        cursor.execute("INSERT INTO heights (rank, athlete, height) VALUES (?, ?, ?)", (rank, name,height))
        connection.commit()
    except AttributeError:
        # This handles missing elements gracefully (e.g., header rows)
        print("Wrong data")
        continue

None None None
1 Tim Forsyth 2.16
1 Takahiro Kimino 2.16
3 Xu Xiaodong 2.16
3 Clifford van Reed 2.16
5 Sven Ootjers 2.16
6 Stanley Osuide 2.13
6 Kristofer Lamos 2.13
6 Antoine Burke 2.13
6 Coenraad Roux 2.13
10 Kim Tae-hoi 2.13
11 Dejan Miloševic 2.13
12 Kostas Liapis 2.13
13 Hugo Muñoz 2.13
14 Giorgio Florindi 2.05
15 Oskari Frösén 2.05
16 Ignacio Pérez 2.00


In [130]:
#verify by querying the databasee
connection = sqlite3.connect("1992_jump_results.db")
cur = connection.cursor()
query = 'select * from heights'
cur.execute(query) 
rows = cur.fetchall()
for row in rows:
    print(row)
connection.commit()

(1, None, None, None)
(2, '1', 'Tim Forsyth', '2.16')
(3, '1', 'Takahiro Kimino', '2.16')
(4, '3', 'Xu Xiaodong', '2.16')
(5, '3', 'Clifford van Reed', '2.16')
(6, '5', 'Sven Ootjers', '2.16')
(7, '6', 'Stanley Osuide', '2.13')
(8, '6', 'Kristofer Lamos', '2.13')
(9, '6', 'Antoine Burke', '2.13')
(10, '6', 'Coenraad Roux', '2.13')
(11, '10', 'Kim Tae-hoi', '2.13')
(12, '11', 'Dejan Miloševic', '2.13')
(13, '12', 'Kostas Liapis', '2.13')
(14, '13', 'Hugo Muñoz', '2.13')
(15, '14', 'Giorgio Florindi', '2.05')
(16, '15', 'Oskari Frösén', '2.05')
(17, '16', 'Ignacio Pérez', '2.00')
